In [25]:
from final_data import get_final_data
from functions import percentage_within_threshold

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
data = get_final_data()

/Users/jennifermei/Documents/GitHub/PIC16B_Politics/final_data.py:50: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_data = df_data[~remove_ca44]


In [43]:
data = data.dropna(axis=0)
data.head()

,Year,State,State Abbr,State FIPS,District,Democratic %,Republican %,Voter Turnout %,Median Age,Median income,...,Less than HS %,HS grad %,Some college %,Bachelor's degree %,Graduate degree %,Below poverty line %,Income under 25k %,Income 25k to 50k %,Income 50k to 75k %,Income above 75k %
0,2014,Alabama,AL,1,1,31.71,68.16,28.48,38.7,23092,...,9.74,21.99,20.23,9.87,5.09,10.38,60.15,22.48,9.76,7.61
1,2014,Alabama,AL,1,2,32.56,67.34,31.59,37.8,22525,...,11.34,21.55,19.61,8.86,5.36,10.34,59.82,23.11,9.69,7.38
2,2014,Alabama,AL,1,3,33.72,66.12,28.89,37.7,21508,...,11.87,21.44,19.35,8.00,5.23,14.25,61.61,22.06,9.55,6.77
3,2014,Alabama,AL,1,4,0.00,98.57,25.38,40.5,20768,...,14.21,23.01,20.46,6.54,4.07,18.83,63.63,22.21,8.59,5.57
4,2014,Alabama,AL,1,5,0.00,74.42,28.23,38.8,25783,...,9.65,18.43,19.82,12.44,7.32,11.20,56.04,21.81,10.48,11.67


In [62]:
# features for training
train_columns = ['Year', 'State FIPS', 'District', 'Voter Turnout %', 'Median Age', 'Median income',
                 'Male Population %', 'Female Population %', 'White %', 'White (non Hispanic or Latino) %',
                 'Hispanic or Latino %', 'Black %', 'American Indian %', 'Asian %', 'Pacific %', 'Multiracial %',
                 'Less than HS %', 'HS grad %', 'Some college %', "Bachelor's degree %", "Graduate degree %",
                 'Below poverty line %', 'Income under 25k %', 'Income 25k to 50k %', 'Income 50k to 75k %',
                 'Income above 75k %']

In [63]:
# features for training
X = data[data['Year'] != 2020][train_columns].values

# target variables
y_dem = data[data['Year'] != 2020]['Democratic %'].values
y_rep = data[data['Year'] != 2020]['Republican %'].values

# normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [64]:
# 80% training, 20% validation
X_train, X_val, y_train_dem, y_val_dem, y_train_rep, y_val_rep = train_test_split(
    X, y_dem, y_rep, test_size=0.2, random_state=42
)

In [65]:
# define and train random forest
rf_dem = RandomForestRegressor(random_state=42)
rf_dem.fit(X_train, y_train_dem)

rf_rep = RandomForestRegressor(random_state=42)
rf_rep.fit(X_train, y_train_rep)

RandomForestRegressor(random_state=42)

In [66]:
# predict on the validation sets
predict_dem_rf = rf_dem.predict(X_val)
predict_rep_rf = rf_rep.predict(X_val)

# if predicting greater than 100, clip it
predict_dem_rf = np.clip(predict_dem_rf, 0, 100)
predict_rep_rf = np.clip(predict_rep_rf, 0, 100)

In [67]:
# evaluate model performance using MSE
mse_dem = mean_squared_error(y_val_dem, predict_dem_rf)
mse_rep = mean_squared_error(y_val_rep, predict_rep_rf)

In [70]:
# print accuracy within 5% of actual and 10% of actual
acc_5_dem = percentage_within_threshold(y_val_dem, predict_dem_rf, threshold_percent=5)
acc_10_dem = percentage_within_threshold(y_val_dem, predict_dem_rf, threshold_percent=10)
acc_5_rep = percentage_within_threshold(y_val_rep, predict_rep_rf, threshold_percent=5)
acc_10_rep = percentage_within_threshold(y_val_rep, predict_rep_rf, threshold_percent=10)

print("Using Random Forest: \n")
print("Mean Squared Error for Democratic % on Validation Set:", mse_dem)
print(f"Percentage of predictions within 5% for Democratic %: {acc_5_dem:.2f}%")
print(f"Percentage of predictions within 10% for Democratic %: {acc_10_dem:.2f}% \n")
print("Mean Squared Error for Republican % on Validation Set:", mse_rep)
print(f"Percentage of predictions within 5% for Republican %: {acc_5_rep:.2f}%")
print(f"Percentage of predictions within 10% for Republican %: {acc_10_rep:.2f}%")

Using Random Forest: 

Mean Squared Error for Democratic % on Validation Set: 247.1330302277992
Percentage of predictions within 5% for Democratic %: 20.08%
Percentage of predictions within 10% for Democratic %: 40.93% 

Mean Squared Error for Republican % on Validation Set: 231.2745419953668
Percentage of predictions within 5% for Republican %: 25.10%
Percentage of predictions within 10% for Republican %: 42.86%
